# Symbolic Neural Modeling

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/pyglove/blob/main/docs/notebooks/ml/neural_modeling.ipynb)

This notebook illustrates how to use PyGlove to symbolically manipulate Keras layers for neural modeling.

In [ ]:
!pip install pyglove

## Symbolizing Keras Layers

Before we can manipulate the combination of Keras layers, we symbolize Keras layer classes via `pg.symbolize`.

In [ ]:
import tensorflow as tf
import pyglove as pg

# Symbolize Keras layers.
Sequential = pg.symbolize(tf.keras.Sequential)
Conv2D = pg.symbolize(tf.keras.layers.Conv2D)
Dense = pg.symbolize(tf.keras.layers.Dense)
Flatten = pg.symbolize(tf.keras.layers.Flatten)
ReLU = pg.symbolize(tf.keras.layers.ReLU)

## Creating a Symbolic Model

By using the symbolic layer classes, we can create a symbolic neural model for 2D image classification with 10 classes.

In [ ]:
def create_model():
  return Sequential([
     Conv2D(16, (5, 5)),
     ReLU(),
     Conv2D(32, (3, 3)),
     ReLU(),
     Flatten(),
     Dense(10)
  ])

model = create_model()

# The symbolized Keras layers can be printed in human readable form.
# For clarity, we hide the default values of the layers.
print(model.format(hide_default_values=True))

## Manipulating Models

What if we want to upscale the model by increasing the number of filters by 2?

In [ ]:
def double_width(k, v, p):
  """A rebind rule for doubling the filters for Conv2D layers.
  
  Args:
    k: A `pg.KeyPath` object representing the location of current node.
    v: The value of current node.
    p: The parent of current node.

  Returns:
    The output value for current node.
  """
  if isinstance(p, Conv2D) and k.key == 'filters':
    return 2 * v
  return v

# Rebind allows the users to manipulate a symbolic object by
# rules.
print(model.rebind(double_width).format(hide_default_values=True))

What if we want to remove the `ReLU` activations?

In [ ]:
def remove_activations(k, v, p):
  if isinstance(v, ReLU):
    # `pg.MISSING_VALUE` is a placeholder for deleting a value from a container.
    return pg.MISSING_VALUE
  return v
print(model.rebind(remove_activations).format(hide_default_values=True))

What if we want to change the number of classes for the classification head from 10 to 100?

In [ ]:
# Query the last Dense layer in the model, and modify its units to 100.
result = pg.query(model, where=lambda v: isinstance(v, Dense))
classification_head_location = list(result.keys())[-1]
model.rebind({
    f'{classification_head_location}.units': 100
})
print(model.format(hide_default_values=True))